# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [54]:
#!pip install selenium

In [55]:
from selenium import webdriver

from selenium.webdriver.common.keys import Keys

# Passo 1: Pegar a cotação do dólar
# abrir o navegador
navegador = webdriver.Chrome()
# entrar no google
navegador.get("https://www.google.com.br/")
# pesquisar cotação dola no google
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dolar")

navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

                                                                                        

# pegar a cotação
cotacao_dolar = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)
#Passo 2: Pegar a cotação do euro
navegador.get("https://www.google.com.br/")
# pesquisar cotação dola no google
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")

navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

                                                                                        

# pegar a cotação
cotacao_euro = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_euro)

# Passo 3: Pegar a cotação do ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje')

cotacao_ouro = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',','.')

print(cotacao_ouro)

navegador.quit()

5.2591
5.3271999999999995
293.81


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

- Atualizando os preços e o cálculo do Preço Final

In [56]:
# Passo 4: Atualizar a base de dados

import pandas as pd

tabela = pd.read_excel('Produtos.xlsx')
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


### Agora vamos exportar a nova base de preços atualizada

In [57]:
# Passo 5: Recalcular os preços

# Atualizar as cotações

tabela.loc[tabela["Moeda"]=="Dólar","Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"]=="Euro","Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"]=="Ouro","Cotação"] = float(cotacao_ouro)



# Preço de compra = Preço original * cotação
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]


# Preço de venda = Preço de compra * Margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]


display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.2591,5259.047409,1.40,7362.666373
1,Carro Renault,4500.00,Euro,5.3272,23972.400000,2.00,47944.800000
2,Notebook Dell,899.99,Dólar,5.2591,4733.137409,1.70,8046.333595
3,IPhone,799.00,Dólar,5.2591,4202.020900,1.70,7143.435530
4,Carro Fiat,3000.00,Euro,5.3272,15981.600000,1.90,30365.040000
5,Celular Xiaomi,480.48,Dólar,5.2591,2526.892368,2.00,5053.784736
6,Joia 20g,20.00,Ouro,293.8100,5876.200000,1.15,6757.630000


In [58]:
# Passo 6: Exportar a base de dados
tabela.to_excel("Produto Novo.xlsx", index=False)
